In [59]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.rc('font', family='SimHei', size=13)
from tqdm import tqdm_notebook

import os,gc,re,warnings,sys
warnings.filterwarnings("ignore")

# Download Dataset

In [4]:
!wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231785/underexpose_train.zip

--2020-05-20 08:24:57--  http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231785/underexpose_train.zip
Resolving tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com (tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com)... 47.253.30.83
Connecting to tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com (tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com)|47.253.30.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284811120 (272M) [application/zip]
Saving to: ‘underexpose_train.zip’

underexpose_train.z 100%[===================>] 271.62M  15.0MB/s    in 20s     

2020-05-20 08:25:17 (13.5 MB/s) - ‘underexpose_train.zip’ saved [284811120/284811120]



In [5]:
!wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231785/underexpose_test.zip

--2020-05-20 08:25:17--  http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231785/underexpose_test.zip
Resolving tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com (tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com)... 47.253.30.83
Connecting to tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com (tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com)|47.253.30.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3433621 (3.3M) [application/zip]
Saving to: ‘underexpose_test.zip’

underexpose_test.zi 100%[===================>]   3.27M  5.25MB/s    in 0.6s    

2020-05-20 08:25:18 (5.25 MB/s) - ‘underexpose_test.zip’ saved [3433621/3433621]



In [6]:
!unzip underexpose_train.zip
!unzip underexpose_test.zip

Archive:  underexpose_train.zip
   creating: underexpose_train/
  inflating: underexpose_train/.DS_Store  
 extracting: underexpose_train/underexpose_train_click-2.zip  
 extracting: underexpose_train/underexpose_train_click-3.zip  
 extracting: underexpose_train/underexpose_train_click-1.zip  
  inflating: underexpose_train/underexpose_item_feat.csv  
 extracting: underexpose_train/underexpose_train_click-4.zip  
  inflating: underexpose_train/underexpose_user_feat.csv  
 extracting: underexpose_train/underexpose_train_click-5.zip  
  inflating: underexpose_train/underexpose_train_click-0.csv  
 extracting: underexpose_train/underexpose_train_click-7.zip  
 extracting: underexpose_train/underexpose_train_click-6.zip  
 extracting: underexpose_train/underexpose_train_click-8.zip  
 extracting: underexpose_train/underexpose_train_click-9.zip  
Archive:  underexpose_test.zip
   creating: underexpose_test/
  inflating: underexpose_test/.DS_Store  
 extracting: underexpose_test/underexpose

# 1. Preprocessing Data

In [2]:
train_user_df = pd.read_csv('./underexpose_train/underexpose_user_feat.csv', names=['user_id','user_age_level','user_gender','user_city_level'])
train_item_df = pd.read_csv('./underexpose_train/underexpose_item_feat.csv')
train_click_0_df = pd.read_csv('./underexpose_train/underexpose_train_click-0.csv',names=['user_id','item_id','time'])

In [68]:
test_click_0_df = pd.read_csv('./underexpose_test/underexpose_test_click-0/underexpose_test_click-0.csv', names=['user_id','item_id','time'])

In [3]:
train_item_df.columns = ['item_id'] + ['txt_vec'+str(i) for i in range(128)] + ['img_vec'+str(i) for i in range(128)]
train_item_df['txt_vec0'] = train_item_df['txt_vec0'].apply(lambda x:float(x[1:]))
train_item_df['txt_vec127'] = train_item_df['txt_vec127'].apply(lambda x:float(x[:-1]))
train_item_df['img_vec0'] = train_item_df['img_vec0'].apply(lambda x:float(x[1:]))
train_item_df['img_vec127'] = train_item_df['img_vec127'].apply(lambda x:float(x[:-1]))

# rank
train_click_0_df['rank'] = train_click_0_df.groupby(['user_id'])['time'].rank(ascending=False).astype(int)

# click cnts
train_click_0_df['click_cnts'] = train_click_0_df.groupby(['user_id'])['time'].transform('count')

In [25]:
train_item_df.head(5)

,item_id,txt_vec0,txt_vec1,txt_vec2,txt_vec3,txt_vec4,txt_vec5,txt_vec6,txt_vec7,txt_vec8,...,img_vec118,img_vec119,img_vec120,img_vec121,img_vec122,img_vec123,img_vec124,img_vec125,img_vec126,img_vec127
0,67898,-2.002905,-0.929881,0.790017,-1.380895,-0.510463,-1.810096,1.363962,0.497401,-4.038903,...,-0.538330,-2.620164,1.277195,0.601015,-0.345312,0.993457,1.351633,2.162675,2.768597,-0.937197
1,66446,4.221673,-1.497139,1.133570,-2.745607,-4.197045,-0.542392,-1.396256,1.838419,-6.066454,...,-4.582711,-1.056910,-2.568084,-2.038061,2.508719,-0.764789,-0.657116,3.252782,2.687366,0.844332
2,63651,2.657970,-0.941863,1.121529,-5.109496,-0.279041,-0.351968,-1.086983,2.703607,-6.494977,...,-0.487683,-1.889119,0.943015,-2.834418,1.633184,2.001801,-2.333152,2.645595,2.280233,-0.694448
3,46824,3.192195,-1.936676,1.199909,-2.562152,-2.573456,0.575841,-2.358653,1.620844,-4.302936,...,-0.621475,-2.091410,0.501600,-3.083864,-1.060091,2.053600,-2.025008,2.399251,2.562317,0.694134
4,54655,1.641749,-1.011634,1.384040,-3.133104,-2.391770,3.766695,-0.907111,-0.560001,-3.468522,...,0.384647,-3.800584,-1.892834,-3.747731,-0.024840,0.601698,-1.463611,2.523813,1.568262,-0.235627


In [4]:
train_click_0_df.head(5)

,user_id,item_id,time,rank,click_cnts
0,4965,18,0.983763,27,29
1,20192,34,0.983772,17,20
2,30128,91,0.983780,23,28
3,29473,189,0.983930,5,20
4,10625,225,0.983925,6,37


In [5]:
item_count = train_click_0_df.groupby(['item_id'])['time'].transform('count').sort_values(ascending=False)

In [10]:
item_count

124454    215
212222    215
212230    215
212229    215
168332    215
         ... 
80736       1
91250       1
169967      1
31232       1
144371      1
Name: time, Length: 241784, dtype: int64

In [6]:
train_item_id = list(train_item_df["item_id"])

In [38]:
def get_possible_negative_item(item_count, threshold=10, given_item=None):
    # Choose the item with click larger than the given threshold
    # Combine the list with all items in train_item csv 
    # Return the list of index and its distribution
    negative_item_list = list(item_count[item_count > 10].index)
    negative_with_chosen_item = list(set(given_item + negative_item_list))
    negative_dist = np.ones(len(negative_with_chosen_item))
    for i, idx in enumerate(negative_with_chosen_item):
        if idx in item_count:
            negative_dist[i] = item_count[idx]
    negative_dist /= sum(negative_dist)
    return negative_with_chosen_item, negative_dist

def negative_sample(possible_negative_items, pni_dist, sample_amount=5):
    random_sample = np.random.choice(possible_negative_items, sample_amount, replace=False, p=pni_dist)
    return random_sample

In [39]:
possible_neg_item, neg_dist = get_possible_negative_item(item_count, given_item=train_item_id)

In [34]:
len(possible_neg_item)
len(neg_dist)

140864

In [41]:
negative_sample(possible_neg_item, neg_dist)

array([149812, 168408, 190327, 205300,  63823])

In [62]:
training_data = []
pos_user_item = train_click_0_df[["user_id", "item_id"]]
for pair in tqdm_notebook(pos_user_item.to_numpy()):
    userId = pair[0]
    pos_pair = (userId, pair[1], 1)
    # Get Negative pair (User Id, Item Id, 0)
    negative_item = negative_sample(possible_neg_item, neg_dist, 5)
    negative_pairs = [(userId, itemId, 0) for itemId in negative_item]
    training_data.append(pos_pair)
    training_data.extend(negative_pairs)

In [63]:
np.random.shuffle(training_data)

In [66]:
with open("train.rating", 'w') as f:
    for pair in training_data:
        f.write("{}\t{}\t{}\n".format(pair[0], pair[1], pair[2]))

In [74]:
test_data, neg_test_samples = [], []
test_user_item = test_click_0_df[["user_id", "item_id"]]
for pair in tqdm_notebook(test_user_item.to_numpy()):
    test_pair = (pair[0], pair[1], 1)
    test_data.append(test_pair)
    
    neg_test_items = negative_sample(possible_neg_item, neg_dist, 99)
    neg_test_sample = [tuple(pair), neg_test_items]
    neg_test_samples.append(neg_test_sample)

In [83]:
with open("test.rating", 'w') as f:
    for pair in test_data:
        f.write("{}\t{}\t{}\n".format(pair[0], pair[1], pair[2]))

with open("test.negative", 'w') as f:
    for l in neg_test_samples:
        negativeItems = "\t".join([str(n) for n in l[1]])
        f.write("({},{})\t{}\n".format(l[0][0], l[0][1], negativeItems))

In [82]:
print("\t".join([str(n) for n in l[1]]))

204881	72817	124904	30389	106493	27590	45482	104732	226255	146820	39410	31158	87335	183202	231572	69372	16597	19606	67350	88705	14465	106915	183662	38028	116433	38925	153774	104777	193372	220057	173288	28503	129086	166047	189598	75962	78542	73453	196902	161645	127490	227218	176821	142624	31122	105786	6645	57476	121730	10724	103923	126400	67084	55645	102363	96782	12552	178140	25480	87025	229799	178206	21571	29405	225873	130499	36753	27425	118183	87703	105174	89348	159720	162023	56620	32876	152626	98904	135955	128943	100706	32489	33195	120703	118313	89898	3182	73631	183758	46094	47021	161492	239493	104390	82588	190315	241645	150892	85207
